In [1]:
from pynq import Overlay
from pynq.lib import AxiGPIO
from pynq import Interrupt
from time import sleep
import asyncio
ol = Overlay("block_design_top.bit")

input_to_lock = AxiGPIO(ol.ip_dict['axi_gpio_0']).channel2
output_from_lock = AxiGPIO(ol.ip_dict['axi_gpio_0']).channel1

input_to_lock.setdirection("out")
input_to_lock.setlength(8)
output_from_lock.setdirection("in")
output_from_lock.setlength(8)

correct_bits_count_line = input_to_lock[0:3]
error_line = input_to_lock[4]
locked_line = input_to_lock[5]
new_seq_ack_line = input_to_lock[6]
sequence_correct = input_to_lock[7]

new_seq_ready = Interrupt('top_0/new_seq_ready')
lock = Interrupt('top_0/lock')
rst = Interrupt('input_interface_0/rst')

current_sequence = 0
secret_sequence = 0
current_correctly_set_bits = 0 # link this to the respective gpio lines
waiting_for_new_secret = True

def init():
    global waiting_for_new_secret
    global secret_sequence
    global current_sequence
    global current_correctly_set_bits
    secret_sequence = 0
    current_sequence = 0
    current_correctly_set_bits = 0
    correct_bits_count_line.write(current_correctly_set_bits)
    waiting_for_new_secret = True
    locked_line.on()
    new_seq_ack_line.off()
    ol.axi_intc_0.register_map.IER = 0xF
    ol.axi_intc_0.register_map.MER = 3
    return

init()

In [2]:
def get_bit(num, index):
    return (num >> index) & 1

def truncate(num, index):
    return num & ((1 << (index + 1)) - 1)

def pulse(line):
    line.on() # TODO: see if there's a better way to do this
    sleep(0.5)
    line.off()

def find_common_sequence():
    global secret_sequence
    global current_sequence
    global current_correctly_set_bits
    for i in range(0, current_correctly_set_bits + 1):
        truncated_secret = truncate(secret_sequence, current_correctly_set_bits - i)
        truncated_seq = current_sequence >> i
        print(truncated_seq == truncated_secret)
        if truncated_seq == truncated_secret:
            current_sequence = truncated_secret
            current_correctly_set_bits = i
            return
    current_sequence = 0
    current_correctly_set_bits = 0

In [3]:
async def wait_for_new_sequence():
    while True:
        await new_seq_ready.wait()
        new_sequence = output_from_lock.read()
        pulse(new_seq_ack_line)
        handle_new_sequence(new_sequence)

def handle_new_sequence(new_sequence):
    global current_correctly_set_bits
    global secret_sequence
    global waiting_for_new_secret
    global current_sequence
    current_sequence = new_sequence
    if not waiting_for_new_secret:
        check_sequence(current_sequence)
        print('entered: {}   bits correct: {}'.format(bin(current_sequence), current_correctly_set_bits))
    else:
        current_correctly_set_bits = current_correctly_set_bits + 1
        if current_correctly_set_bits == 8:
            current_correctly_set_bits = 0
            secret_sequence = current_sequence
            waiting_for_new_secret = False
            current_sequence = 0
            locked_line.off()
            print('set new secret sequence to: {}'.format(bin(secret_sequence)))
        else:
            print('entered: {}   next editing bit: {}'.format(bin(current_sequence), current_correctly_set_bits))
        correct_bits_count_line.write(current_correctly_set_bits)
        
async def set_new_sequence():
    while True:
        await lock.wait()
        init()
        print('setting new secret')

async def reset():
    while True:
        await rst.wait()
        init()
        print('reset issued, awaiting new secret')
    
def check_sequence(sequence):
    global current_correctly_set_bits
    next_bit = current_correctly_set_bits
    if get_bit(sequence, next_bit) == get_bit(secret_sequence, next_bit):
        current_correctly_set_bits += 1
        if current_correctly_set_bits == 8:
            current_correctly_set_bits = 0
            signal_unlock()
    else:
        # reset to longest common sequence between suffix and prefix
        find_common_sequence()
        raise_error()
    correct_bits_count_line.write(current_correctly_set_bits)

def raise_error():
    error_line.on()
    sleep(1)
    error_line.off()

def signal_unlock():
    sequence_correct.on()
    sleep(1)
    sequence_correct.off()

async def main():
    task1 = asyncio.create_task(wait_for_new_sequence())
    task2 = asyncio.create_task(set_new_sequence())
    task3 = asyncio.create_task(reset())
    await task1
    await task2
    await task3

In [4]:
print(ol.axi_intc_0.register_map.ISR)
print(ol.axi_intc_0.register_map.IPR)
print(ol.axi_intc_0.register_map.IER)
ol.axi_intc_0.register_map.MER

0x0
0x0
0x7


Register(ME=1, HIE=1)

In [ ]:
asyncio.run(main())

entered: 0b0   next editing bit: 1
entered: 0b0   next editing bit: 2
entered: 0b100   next editing bit: 3
entered: 0b1100   next editing bit: 4
entered: 0b11100   next editing bit: 5
entered: 0b111100   next editing bit: 6
entered: 0b1111100   next editing bit: 7
set new secret sequence to: 0b11111100
entered: 0b0   bits correct: 1
entered: 0b0   bits correct: 2
False
True
entered: 0b0   bits correct: 1
entered: 0b0   bits correct: 2
setting new secret
entered: 0b0   next editing bit: 1
entered: 0b10   next editing bit: 2
entered: 0b110   next editing bit: 3
entered: 0b110   next editing bit: 4
entered: 0b10110   next editing bit: 5
entered: 0b10110   next editing bit: 6
entered: 0b10110   next editing bit: 7
set new secret sequence to: 0b10110
entered: 0b0   bits correct: 1
entered: 0b10   bits correct: 2
entered: 0b110   bits correct: 3
entered: 0b110   bits correct: 4
entered: 0b10110   bits correct: 5
False
False
False
True
entered: 0b110   bits correct: 3
False
False
False
False
